# 任务一报告

### 网络目的：

- 训练一个既能识别mnist数据集又能识别自己手写数字的神经网络。

### 网络面临状况：

- mnist数据集有 *70000* 个样本，自己手写只有 *3000* 个样本。

### 解决思路：

- 使网络的评价重心放在测试自己手写的 *3000* 样本上，**并优先选用LeNet模型进行测试**，后根据自己测试集的**精确率（Accuracy）**来进行网络超参的调整。

### 网络超参：

- 层数、每一层卷积核个数、卷积权重初始化方式、dropout ratio、BN（批量归一化）、全连接层神经元个数、激活函数的选择、优化器的选择等。

### 数据集按比例分配如下：

- mnist训练集+测试集 = *60000* + *10000*，自己手写训练集+测试集 = *2571* + *429* (*6 : 1*)，整合训练集+测试集 = *62571* + *10429*

### 网络调试记录(以下准确率未特殊说明均为在自己测试集上的准确率)

1. LeNet模型（两层卷积层+一层全连接+softmax）精确率约 *0.95* 


2. 以全局池化代替全连接层 *0.93* 左右精确率


3. 尝试添加 BN 层，发现在全局池化下精确率增加到 *0.96* 左右，在全连接层下精确率 *0.93* 左右


4. 加入 dropout 层需要更多迭代模型验证 loss 才会收敛，同时使得模型在测试集准确率更高。


5. 但将 dropout 和 BN 层两者结合起来的话，精确率峰值大概在迭代 5 次模型有 *0.92* 左右精确率，此后随迭代次数增加而减少，在迭代到 *20* 次后在自己的测试集上精确率变为 *0.7* 。而在mnist测试集上精确率为 *0.97* 左右，推测两者的结合在此模型里对模型在不同数据集上的泛化能力有抑制作用。


6. **最佳组合**：全局池化+dropout迭代 *10* 次稳定 *0.97* 左右精确率

### 综上，最终网络结构如下：

![model map](fig/multi_model_n.png)



网络配置：
`model_n.compile(optimizer='rmsprop',loss='sparse_categorical_crossentropy', metrics=['accuracy'])`



查看训练验证的精度，损失图（验证集占训练集的0.14）：
![acc and loss](fig/1loss.png)

### 网络精确率：
- 在mnist测试集上的精确率：*0.9897*

- 在自己手写数字测试集精确率为：*0.9720*

- 在总测试集上的精确率为：*0.9889*

### 测试集可视化
查看网络在部分测试集上预测正确的示例：

![good pred](fig/1good_pred.png)

查看网络在部分测试集上预测错误的示例：

![wrong pred](fig/1wrong_pred.jpg)

混淆矩阵：
![confusion_matrix](fig/1cm.png)


**由于模型在mnist和自己手写数字的测试集精确率都超过 *0.97* ，且可以发现有些错误预测数字连本人或许都难以识别，可以认为这种网络比LeNet在自己手写数字上更具有泛化能力，且基本可以识别我们的手写数字。**

---------------------------------------------------------

### 为证明这种网络对自己手写数据更具有泛化能力，将原 LeNet 与此网络作如下对比：

1. **不使用自己手写数字作为训练集，仅用mnist数据集进行训练,使用自己数据集测试**


- 使用LeNet测试，在自己手写测试集上精确率约 *0.40* ，预测结果较差。


- 使用自己的网络测试，在自己手写测试集上精确率约 *0.63* ，其中有约 *1/4* 预测失误是数字 0 ,大部分预测成 12 或 6 等易混淆的数字。


 **粗略得出结论：LeNet在mnist数据集上过拟合，所以需要一些防止过拟合的措施**

（所有测试并没有直接关系）
- 使用LeNet，再添加一卷积层，并用全局池化代替全连接层 在自己手写测试集上精确率约 *0.50* 。


- 使用LeNet并用全局池化代替全连接层，精确率约 *0.55*。尝试过改变学习率，或改变优化器为 Adam（原RMSprop），或添加L2正则化，无明显效果。使用

    dropout层后，loss下降趋势明显，迭代 *10* 次后精确率稳定在 *0.63* 左右。又尝试再添加一层卷积，发现即使增大dropout率，精确率下降，且loss下降后又重

    新升高，过拟合，舍弃。后改用卷积核初始化方式为 Xavier 正态分布，结果精确率可以达到 *0.68* 。之后又改为全连接层后结果越来越差。  


2. **只使用自己数据集来训练，用mnist数据集测试。**


- 自己的网络在自己手写测试集精确率约 *0.95*，但在mnist所有数据集的精确率只有 *0.35* 左右


- LeNet 自己手写测试集精确率约 *0.80* 而在mnist所有数据上精确率也只有 *0.30* 左右

**再得出结论：经改良的LeNet在泛化能力和拟合能力较强，可以适应更多数据。**

# 问题提出


### 1.模型精确率提升细微例如从 0.95 到 0.97 是否可以判段新模型更好，或者说判定模型更好的标准是否只有指标 ？

可以认为更好

---

### 2.判段一个模型已经完善的必要条件包括验证集的 loss 收敛吗？

对

---

### 3.权重初始化问题是如何影响模型的，为什么特定的权重初始化可以防止陷入局部最优解？

权重初始化很重要。

---

### 4.dropout层更适应全连接层？BN层更适应卷积层？两者简单结合为何有时导致模型更差？

对

---

### 5.网格搜索和k折交叉验证方法在什么时候更好用

网格搜索基本用不到

---


# 技巧摘录

1. 层数、每一层卷积核个数、卷积权重初始化方式、dropout ratio、BN、全连接层神经元个数、Relu等网络结构参数应该如何选？
2. 学习率、decay等solver参数又该如何设置？

- 网络结构设计

    如果我们在Cifar10数据集上进行实验，最好能够找几个在该数据集上效果较好的网络结构，看看别人的网络超参数是如何设置的，有什么共性（比如每一层卷积核的个数、大小、层数等）。原则就是“尽可能采用成熟的网络结构，近可能少改动网络结构”。
    
    
- 实验的输出

    一般情况下，在实验的时候会有，train loss、test loss、训练集准确率、测试集准确率，根据它们的特点，我们大致可以得出网络的状态。训练集准确率高、测试集准确率低，过拟合；训练集准确率低、测试集准确率低，欠拟合；训练集、测试集上的准确率和初始状态相比，没有明显的变化，网络在“徘徊”。
    
    
- “过拟合”
    这时候需要提高网络的泛化能力，例如引入dropout、或者增大已有dropout ration等。
    
    
- “欠拟合”
    这时候需要增强模型的拟合能力，，例如增加网络层数、增加节点数，减少dropout等。

0|1|2|3|4|5|6|7|8|9
--:|--:|--:|--:|--:|--:|--:|--:|--:|--:
0 | 0| 0| 0| 0| 0| 0| 0| 0| 0
0 | 0| 0| 0| 0| 0| 0| 2| 0| 0
0 | 0| 0| 0| 0| 0| 0| 0| 0| 0
0 | 0| 1| 0| 0| 0| 0| 0| 1| 0
1 | 0| 0| 0| 0| 0| 0| 0| 0| 0
0 | 0| 0| 1| 0| 0| 0| 0| 0| 0
0 | 0| 0| 0| 0| 0| 0| 0| 0| 0
0 | 0| 0| 0| 0| 0| 0| 0| 0| 1
1 | 0| 0| 0| 0| 2| 1| 0| 0| 0
0 | 1| 0| 0| 0| 0| 0| 0| 0| 0
